# NLF Fanatasy NN


## Preprocessing

In [52]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MaxAbsScaler
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt


#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("Resources/ADPxFinal.csv")

In [54]:
# Labels Columnns to be droped for training 
col = [0,1,2,3]
# Remove unnamed column, Player, Pos and 2021 Team
df.drop(df.columns[col],axis=1,inplace=True)
df = df.dropna()

In [55]:
# Split our preprocessed data into our features and target arrays
# Target
y = df['Production21'].values
# Features
X = df.drop('Production21', axis=1).values

In [56]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 10)

In [57]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [58]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=14,
        step=2), activation=activation, input_dim=len(X_train_scaled[0])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=14,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [59]:
# Import the kerastuner library
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    project_name="keras_tuner",
    overwrite=True,
    max_epochs=50,
    hyperband_iterations=2)

In [60]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train, y_train, epochs = 50, validation_data=(X_test_scaled,y_test))

Trial 180 Complete [00h 00m 03s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.0
Total elapsed time: 00h 03m 55s
INFO:tensorflow:Oracle triggered exit


In [47]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 3,
 'num_layers': 1,
 'units_0': 3,
 'tuner/epochs': 2,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 3,
 'tuner/round': 0}

In [48]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: -9.1849e-01 - accuracy: 0.0000e+00 - 207ms/epoch - 104ms/step
Loss: -0.9184902906417847, Accuracy: 0.0


In [49]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 141       
                                                                 
 dense_1 (Dense)             (None, 3)                 12        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 157
Trainable params: 157
Non-trainable params: 0
_________________________________________________________________


In [50]:
# Evaluate the model using the test data
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: -9.1849e-01 - accuracy: 0.0000e+00 - 24ms/epoch - 12ms/step
Loss: -0.9184902906417847, Accuracy: 0.0


In [51]:
# Export our model to HDF5 file
best_model.save('NFL_Keras_Optimized.h5')